In [ ]:
import os
while os.getcwd().split('\\')[-1] != 'ecological-networks':
    %cd ..
import setup_paths
setup_paths.add_path()
from Species_Reintroductions import *
DATA_TYPE = 'Real_1D'
CASE_TO_SOLVE_init(DATA_TYPE)
get_CASE()

In [ ]:
def G_iter(data_two):
    for key in list(data_two.keys()):
        for ensembles_ID in range(len(data_two[key])):
            yield (key, ensembles_ID, 0)

def get_ScatterData(NetworkName, loc):
    Data_Scatter = []
    #print(NetworkName)
    M_df = M[NetworkName]
    data_two = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", f"{NetworkName}", f"{NetworkName}-Data_two.pkl"))
    #data_two = UnpickleObj(f"{NetworkName}-Data_two.pkl")
    Keys_Filter = [[0,1,2], [2,5,8], [0,2]]
    Ensembles_batch = [0,10]
    data_two = {k: v[Ensembles_batch[0]:min(Ensembles_batch[1],len(v))] for k,v in data_two.items() if k in set(list(itertools.product(*Keys_Filter)))}
    del Keys_Filter, Ensembles_batch
    
    for approach in ['three','four','five','six','rand']:
        for key, E, T in itertools.islice(G_iter(data_two),0,None,1):
            try:
                X = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '02_Generate_Database', f"{DATA_TYPE}_Data", NetworkName, f"{NetworkName}-Data_{approach}_{key}_{E}_{T}.pkl"))
                M_per = RemovalFromNodeList(data_two[key][E],M_df,axis=key[2])
                M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                
                M_per_dict = {}
                M_per_dict['n'] = M_per.shape[0]
                M_per_dict['m'] = M_per.shape[1]
                M_per_dict['L'] = sum(np.array(M_per).flatten())
                M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
                M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
                M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
                M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                
                Z1 = np.mean(np.array(X[3][0][-1,:]))
                Z2 = np.mean(np.array(list(map(isExtinct,X[3][0][-1,:]))))
                Z3 = getResilience(X[3][0])
                Z4 = getPersistence(np.array(list(map(isExtinct,X[3][0][-1,:])))) / M_df.shape[T]
                
                temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N']] + [Z1,Z2,Z3,Z4]
                Data_Scatter += [temp]
                
                for Step in list(range(1,len(X[0])+1)):
                    M_per = M_per.append(M_df.loc[X[0][Step-1]])
                    M_per[M_per.isnull()] = M_df
                    M_per = M_per.loc[(M_per!=0).any(axis=1)] #Cleaning Row isoloates
                    M_per = M_per.loc[:, (M_per!=0).any(axis=0)] #Cleaning Column isoloates
                    
                    M_per_dict['n'] = M_per.shape[0]
                    M_per_dict['m'] = M_per.shape[1]
                    M_per_dict['L'] = sum(np.array(M_per).flatten())
                    M_per_dict['S'] = M_per_dict['n'] + M_per_dict['m']
                    M_per_dict['A'] = M_per_dict['m'] / M_per_dict['n']
                    M_per_dict['C'] = M_per_dict['L'] / (M_per_dict['m']*M_per_dict['n'])
                    M_per_dict['N'] = NestednessCalculator(np.array(M_per)).nodf(np.array(M_per))
                    
                    Z1 = np.mean(np.array(X[3][Step][-1,:]))
                    Z2 = np.mean(np.array(list(map(isExtinct,X[3][Step][-1,:]))))
                    Z3 = getResilience(X[3][Step])
                    Z4 = getPersistence(np.array(list(map(isExtinct,X[3][Step][-1,:])))) / M_df.shape[T]
                    
                    temp = [M_per_dict[x] for x in ['n','m','L','S','A','C','N']] + [Z1,Z2,Z3,Z4]
                    Data_Scatter += [temp]
            except:
                #print(key, E, T)
                pass
    DF = pd.DataFrame(Data_Scatter, columns = ['n','m','L','S','A','C','N','Abundance','Ob Abundance','Settling Time','Persistence'])
    DF.drop_duplicates(inplace=True, ignore_index=True)
    #print(NetworkName, len(Data_Scatter), DF.shape[0])
    FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', 'ScatterData', f"{loc}", f"{NetworkName}.pkl")
    os.makedirs(os.path.dirname(FileName), exist_ok=True)
    PickleObj(DF,FileName)

In [ ]:
for CASE_TO_SOLVE, loc in zip(['Real_1D', 'Syn_01', 'Syn_2D', 'Real_2D'],['01','02','03','04']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))

    if DATA_TYPE.split('_')[0] == 'Syn':
        M = {k:M[k] for k in M.keys() if int(k.split('_')[2]) == 100}

    with Parallel(n_jobs=-1, verbose = 10) as parallel:
            data = parallel(delayed(get_ScatterData)(NetworkName,loc) for NetworkName in itertools.islice(M.keys(),0,None,1))


In [ ]:
UPDATE_SAVED = False
for CASE_TO_SOLVE, loc in zip(['Real_1D', 'Syn_1D', 'Syn_2D', 'Real_2D'],['01','02','03','04']):
    DATA_TYPE = CASE_TO_SOLVE
    CASE_TO_SOLVE_init(DATA_TYPE)
    NetworkName_Solved_df = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"Network_Solved_{DATA_TYPE.split('_')[0]}.pkl"))
    M = UnpickleObj(os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '01_Simulation_Execution', f"M_{DATA_TYPE.split('_')[0]}.pkl"))

    D = {}
    DF = pd.DataFrame([],columns = ['n','m','L','S','A','C','N','Abundance','Ob Abundance','Settling Time','Persistence'])

    for NetworkName in itertools.islice(M.keys(),0,None,1):
        X = UnpickleObj(f"ScatterData/{loc}/{NetworkName}.pkl")
        X = X.apply(pd.to_numeric)
        X.drop(X[ X['Ob Abundance'] == 0 ].index ,inplace = True)
        Y1 = X.corr(method = 'pearson')
        Y2 = X.corr(method = 'kendall')
        Y3 = X.corr(method = 'spearman')
        D[NetworkName] = [Y1,Y2,Y3]
        DF = pd.concat([DF,X], ignore_index = True)
    del NetworkName, X, Y1, Y2, Y3, M, NetworkName_Solved_df
    DF = DF.apply(pd.to_numeric)
    DF.sort_values(by = ['Abundance', 'Ob Abundance'], axis = 0, inplace = True, ignore_index = True)
    DF.drop(DF[ DF['Ob Abundance'] == 0 ].index ,inplace = True)
    DF.reset_index(drop = True, inplace = True)

    Z = DF.describe()
    Y1 = DF.corr(method = 'pearson')
    Y2 = DF.corr(method = 'kendall')
    Y3 = DF.corr(method = 'spearman')
    Data = [Z, Y1, Y2, Y3, DF, D]
    if UPDATE_SAVED:
        PickleObj(Data, os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_{loc}_Consolidated.pkl"))
del loc, CASE_TO_SOLVE, DF, Z, Y1, Y2, Y3, Data
del D, UPDATE_SAVED

In [ ]:
FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_01_Consolidated.pkl")
D01 = UnpickleObj(FileName)[4]
D01.drop(D01[ D01['N'] == 0 ].index ,inplace = True)
D01.dropna(axis = 0, inplace = True)
D01 = D01[D01['Abundance'] > 1.0]
D01.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_02_Consolidated.pkl")
D02 = UnpickleObj(FileName)[4]
D02.drop(D02[ D02['N'] == 0 ].index ,inplace = True)
D02.dropna(axis = 0, inplace = True)
D02 = D02[D02['Abundance'] > 1.0]
D02.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_03_Consolidated.pkl")
D03 = UnpickleObj(FileName)[4]
D03.drop(D03[ D03['N'] == 0 ].index ,inplace = True)
D03.dropna(axis = 0, inplace = True)
D03 = D03[D03['Abundance'] > 0.3]
D03.reset_index(drop = True, inplace = True)

FileName = os.path.join(os.getcwd(), 'Code', '01_Species_Reintroductions', '03_Scatter_Data', f"Scatter_04_Consolidated.pkl")
D04 = UnpickleObj(FileName)[4]
D04.drop(D04[ D04['N'] == 0 ].index ,inplace = True)
D04.dropna(axis = 0, inplace = True)
D04 = D04[D04['Abundance'] > 0.3]
D04.reset_index(drop = True, inplace = True)